# Imports

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

separator = '\t'
#Eva
#filepath = 'D:/Workspace/_DataMining/DataSets'
#Silke
filepath = 'C:/Users/Silke/Documents/GitHub/DataMiningAP'

# IMDB
## Creation of RDD's and DF's

In [2]:
#TITLE.BASICS
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = sc.textFile(filepath + '/IMDB/titleBasics.tsv')
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l).map(lambda l: l.split(separator))

#create schema for df
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', StringType(), True))
tbFields.append(StructField('startYear', StringType(), True))
tbFields.append(StructField('endYear', StringType(), True))
tbFields.append(StructField('runtimeMinutes', StringType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df = sqlContext.createDataFrame(titleBasics_rdd, tbSchema)

#clean data
titleBasics_df = titleBasics_df.drop('originalTitle', 'runtimeMinutes').sort('tconst', ascending=True)
titleBasics_df.show(5, truncate=True)

+---------+---------+--------------------+-------+---------+-------+--------------------+
|   tconst|titleType|        primaryTitle|isAdult|startYear|endYear|              genres|
+---------+---------+--------------------+-------+---------+-------+--------------------+
|tt0000001|    short|          Carmencita|      0|     1894|     \N|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|      0|     1892|     \N|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      0|     1892|     \N|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|      0|     1892|     \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|      0|     1893|     \N|        Comedy,Short|
+---------+---------+--------------------+-------+---------+-------+--------------------+
only showing top 5 rows



In [10]:
#TITLE.AKAS
#tconst, ordering, title, region, language, types, attributes, isOriginalTitle
titleAkas_data = sc.textFile(filepathEva + '/IMDB/titleAkas.tsv')
titleAkas_rdd = titleAkas_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
taFields = []
taFields.append(StructField('titleId', StringType(), True))
taFields.append(StructField('ordering', StringType(), True))
taFields.append(StructField('title', StringType(), True))
taFields.append(StructField('region', StringType(), True))
taFields.append(StructField('language', StringType(), True))
taFields.append(StructField('types', StringType(), True))
taFields.append(StructField('attributes', StringType(), True))
taFields.append(StructField('isOriginalTitle', StringType(), True))                       

taSchema = StructType(taFields)

#create df
titleAkas_df= sqlContext.createDataFrame(titleAkas_rdd, taSchema)

#clean data
titleAkas_df = titleAkas_df.filter(titleAkas_df.region =='US')
titleAkas_df = titleAkas_df.dropDuplicates(['titleId']).sort('titleId', ascending=True)

titleAkas_df.show(5, truncate=True)

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       3|          Carmencita|    US|      \N|         \N|                  \N|              0|
|tt0000002|       5|The Clown and His...|    US|      \N|         \N|literal English t...|              0|
|tt0000005|       1| Blacksmithing Scene|    US|      \N|alternative|                  \N|              0|
|tt0000006|       3|   Chinese Opium Den|    US|      \N|         \N|                  \N|              0|
|tt0000007|       1|Corbett and Court...|    US|      \N|         \N|                  \N|              0|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
only showing top 5 rows



In [11]:
#TITLE.PRINCIPALS
#tconst, ordering, nconst, category, job, characters
titlePrincipals_data = sc.textFile(filepathEva + '/IMDB/titlePrincipals.tsv')
titlePrincipals_rdd = titlePrincipals_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
tpFields = []
tpFields.append(StructField('tconst', StringType(), True))
tpFields.append(StructField('ordering', StringType(), True))
tpFields.append(StructField('nconst', StringType(), True))
tpFields.append(StructField('category', StringType(), True))
tpFields.append(StructField('job', StringType(), True))
tpFields.append(StructField('characters', StringType(), True))                     

tpSchema = StructType(tpFields)

#create df
titlePrincipals_df = sqlContext.createDataFrame(titlePrincipals_rdd, tpSchema)

#clean data
titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.category.contains("actor") | titlePrincipals_df.category.contains("actress"))
titlePrincipals_df = titlePrincipals_df.drop('job').sort('tconst', ascending=True)

titlePrincipals_df.show(2, truncate = True)

+---------+--------+---------+--------+--------------+
|   tconst|ordering|   nconst|category|    characters|
+---------+--------+---------+--------+--------------+
|tt0000005|       2|nm0653042|   actor| ["Assistant"]|
|tt0000005|       1|nm0443482|   actor|["Blacksmith"]|
+---------+--------+---------+--------+--------------+
only showing top 2 rows



In [12]:
#TITLE.RATINGS
#tconst, averageRating, numVotes
titleRatings_data = sc.textFile(filepathEva + '/IMDB/titleRatings.tsv')
titleRatings_rdd = titleRatings_data.filter(lambda l: 'averageRating' not in l).map(lambda l: l.split(separator))

#create schema for df
trFields = []
trFields.append(StructField('tconst', StringType(), True))
trFields.append(StructField('averageRating', StringType(), True))
trFields.append(StructField('numVotes', StringType(), True))                    

trSchema = StructType(trFields)

#create df
titleRatings_df= sqlContext.createDataFrame(titleRatings_rdd, trSchema)

titleRatings_df.show(2, truncate = True)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.8|    1416|
|tt0000002|          6.4|     167|
+---------+-------------+--------+
only showing top 2 rows



In [13]:
#TITLE.EPISODE
#tconst, parentTconst, seasonNumber, episodeNumber
titleEpisode_data = sc.textFile(filepathEva + '/IMDB/titleEpisode.tsv')
titleEpisode_rdd = titleEpisode_data.filter(lambda l: 'seasonNumber' not in l).map(lambda l: l.split(separator))

#create schema for df
teFields = []
teFields.append(StructField('tconst', StringType(), True))
teFields.append(StructField('parentTconst', StringType(), True))
teFields.append(StructField('seasonNumber', StringType(), True))
teFields.append(StructField('episodeNumber', StringType(), True))                    

teSchema = StructType(teFields)

#create df
titleEpisode_df= sqlContext.createDataFrame(titleEpisode_rdd, teSchema)

titleEpisode_df.show(2, truncate = True)


+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
+---------+------------+------------+-------------+
only showing top 2 rows



In [14]:
#NAME.BASICS
#nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles
nameBasics_data = sc.textFile(filepathEva + '/IMDB/nameBasics.tsv')
nameBasics_rdd = nameBasics_data.filter(lambda l: 'primaryName' not in l).map(lambda l: l.split(separator))

#create schema for df
nmFields = []
nmFields.append(StructField('nconst', StringType(), True))
nmFields.append(StructField('primaryName', StringType(), True))
nmFields.append(StructField('birthYear', StringType(), True))
nmFields.append(StructField('deathYear', StringType(), True))
nmFields.append(StructField('primaryProfession', StringType(), True))
nmFields.append(StructField('knownForTitles', StringType(), True))

nmSchema = StructType(nmFields)

#create df
nameBasics_df= sqlContext.createDataFrame(nameBasics_rdd, nmSchema)

#clean data
nameBasics_df = nameBasics_df.drop('birthYear', 'knownForTitles')
nameBasics_df = nameBasics_df.filter(nameBasics_df.primaryProfession.contains("actor")|nameBasics_df.primaryProfession.contains("actress")).sort('nconst', ascending=True)

nameBasics_df.show(5, truncate = True)

+---------+---------------+---------+--------------------+
|   nconst|    primaryName|deathYear|   primaryProfession|
+---------+---------------+---------+--------------------+
|nm0000001|   Fred Astaire|     1987|soundtrack,actor,...|
|nm0000002|  Lauren Bacall|     2014|  actress,soundtrack|
|nm0000003|Brigitte Bardot|       \N|actress,soundtrac...|
|nm0000004|   John Belushi|     1982|actor,writer,soun...|
|nm0000005| Ingmar Bergman|     2007|writer,director,a...|
+---------+---------------+---------+--------------------+
only showing top 5 rows



## Join title.Basics and title.Akas and title.Ratings

In [15]:
#Aliases
Basics = titleBasics_df.alias('Basics')
Akas = titleAkas_df.alias('Akas')
Ratings = titleRatings_df.alias('Ratings')

#join title.Basics to title.Akas
AkasBasics = Akas.join(Basics, Akas.titleId == Basics.tconst, how='left')

#clean data 1st join
AkasBasics = AkasBasics.filter(AkasBasics.isAdult =='0')
AkasBasics = AkasBasics.drop('ordering', 'language', 'types', 
                               'attributes', 'isOriginalTitle', 'tconst', 
                               'primaryTitle', 'isAdult')
AkasBasics = AkasBasics.filter(AkasBasics.titleType.contains("movie") | AkasBasics.titleType.contains("tvSeries") | AkasBasics.titleType.contains("tvEpisode"))
AkasBasics = AkasBasics.filter(AkasBasics.startYear >='1950').sort('titleId', ascending=True)

#join AkasBasics with title.Ratings
titleABR = AkasBasics.join(Ratings, AkasBasics.titleId == Ratings.tconst, how='left').sort('titleId', ascending=True)

#clean data 2nd join
titleABR = titleABR.drop('tconst')

titleABR.show(3, truncate = True)


+---------+--------------------+------+---------+---------+-------+--------------------+-------------+--------+
|  titleId|               title|region|titleType|startYear|endYear|              genres|averageRating|numVotes|
+---------+--------------------+------+---------+---------+-------+--------------------+-------------+--------+
|tt0021617|   Arizona Territory|    US|    movie|     1950|     \N|             Western|          5.4|      18|
|tt0031603|Made in Germany -...|    US|    movie|     1957|     \N|     Biography,Drama|          6.5|      13|
|tt0035423|    Kate and Leopold|    US|    movie|     2001|     \N|Comedy,Fantasy,Ro...|          6.4|   72053|
+---------+--------------------+------+---------+---------+-------+--------------------+-------------+--------+
only showing top 3 rows



# BabyNames
## RDD Creation
### US

In [3]:
#US Baby Names
#NationalNames.csv
#Id,Name,Year,Gender,Count
nationalname_rdd_init = (sc.textFile(filepath + '/us-baby-names/NationalNames.csv'))
nationalname_rdd = nationalname_rdd_init.filter(lambda x: 'Gender' not in x)
nationalname_rdd = nationalname_rdd.map(lambda x: x.split(","))

### Per State

In [4]:
#State Baby Names
#StateNames.csv
#Id,Name,Year,Gender,State,Count
full_statename_rdd_init = (sc.textFile(filepath + '/us-baby-names/StateNames.csv'))
full_statename_rdd = full_statename_rdd_init.filter(lambda x: 'Gender' not in x).map(lambda x: x.split(","))

## US States

In [5]:
us_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
us_states_abbreviations = list(us_states.keys())

# Analysis
## How many times does name x occur per year

In [6]:
#year = "2005"
#nationalname_peryear_rdd = nationalname_rdd.filter(lambda x: x[2] == year)
name = "Arwen"
pername_nationalname_rdd = nationalname_rdd.filter(lambda x: x[1] == name)
# Create a trace
#print(nationalname_rdd.map(lambda x: int(x[4])).take(10))
#print(nationalname_rdd.map(lambda x: int(x[2])).take(10))

trace = go.Scatter(
    x = pername_nationalname_rdd.map(lambda x: int(x[2])).collect(),
    y = pername_nationalname_rdd.map(lambda x: int(x[4])).collect()
)

data = [trace]

py.iplot(data, filename='basic-line')